In [51]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [52]:
### Load the trained model, scaler pickle, onehot

model = load_model('model.h5')

with open ('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open ('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open ('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [148]:
input_data = [{
'CreditScore': 800,
    'Geography': 'Spain',
    'Gender': 'Male',
    'Age': 35,
    'Tenure': 4,
    'Balance': 10000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 90000},

{'CreditScore':600,
'Geography': 'France',
'Gender':'Male',
'Age':40,
'Tenure':3,
'Balance':60000,
'NumOfProducts':2,
'HasCrCard':1,
'IsActiveMember':1,
'EstimatedSalary':50000},


{
'CreditScore':300,
'Geography': 'Germany',
'Gender':'Female',
'Age':20,
'Tenure':5,
'Balance':20000,
'NumOfProducts':2,
'HasCrCard':0,
'IsActiveMember':0,
'EstimatedSalary':8000
},{
'Geography': 'Spain',
'Gender': 'Female',
'Age': 60,
'Tenure': 8,
'Balance': 10000,
'NumOfProducts': 2,
'HasCrCard': 0,
'IsActiveMember': 1,
'EstimatedSalary': 6000},
]

# print(input_data)

In [149]:
input_df= pd.DataFrame(input_data)
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800.0,Spain,Male,35,4,10000,2,1,1,90000
1,600.0,France,Male,40,3,60000,2,1,1,50000
2,300.0,Germany,Female,20,5,20000,2,0,0,8000
3,NaN,Spain,Female,60,8,10000,2,0,1,6000


In [150]:
geo_encoder = onehot_encoder_geo.transform(input_df[['Geography']]).toarray()
geo_encoder

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [151]:
# geo_encoded = onehot_encoder_geo.get_feature_names_out([[input_data['Geography']]])
geo_encoder_df = pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [152]:
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0
3,0.0,0.0,1.0


In [153]:
# input_df=pd.DataFrame([input_data])
# input_df

In [154]:
# input_df = pd.concat([input_df.reset_index(drop=True),geo_encoder_df],axis=1)

In [155]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800.0,Spain,Male,35,4,10000,2,1,1,90000
1,600.0,France,Male,40,3,60000,2,1,1,50000
2,300.0,Germany,Female,20,5,20000,2,0,0,8000
3,NaN,Spain,Female,60,8,10000,2,0,1,6000


In [156]:
input_df['Gender']= label_encoder_gender.transform(input_df['Gender'])


In [157]:
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoder_df],axis=1)


In [158]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,800.0,1,35,4,10000,2,1,1,90000,0.0,0.0,1.0
1,600.0,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0
2,300.0,0,20,5,20000,2,0,0,8000,0.0,1.0,0.0
3,NaN,0,60,8,10000,2,0,1,6000,0.0,0.0,1.0


In [159]:
scaled_input = scaler.transform(input_df)
print(scaled_input)


# input_scaled=scaler.transform(input_df)
# print(input_scaled)

[[ 1.53956105e+00  9.13247552e-01 -3.70568589e-01 -3.48369103e-01
  -1.05836066e+00  8.08436154e-01  6.49202671e-01  9.74816989e-01
  -1.81365396e-01 -9.98501123e-01 -5.79467227e-01  1.73494238e+00]
 [-5.35985161e-01  9.13247552e-01  1.04793593e-01 -6.95393486e-01
  -2.57811188e-01  8.08436154e-01  6.49202671e-01  9.74816989e-01
  -8.76832208e-01  1.00150113e+00 -5.79467227e-01 -5.76388018e-01]
 [-3.64930447e+00 -1.09499335e+00 -1.79665513e+00 -1.34471949e-03
  -8.98250767e-01  8.08436154e-01 -1.54035103e+00 -1.02583358e+00
  -1.60707236e+00 -9.98501123e-01  1.72572313e+00 -5.76388018e-01]
 [            nan -1.09499335e+00  2.00624232e+00  1.03972843e+00
  -1.05836066e+00  8.08436154e-01 -1.54035103e+00  9.74816989e-01
  -1.64184570e+00 -9.98501123e-01 -5.79467227e-01  1.73494238e+00]]


In [160]:
### Predict Churn

prediction = model.predict(scaled_input)
print(prediction)

# prediction1=model.predict(input_scaled)
# print(prediction1)



1/1 [==============================] - 0s 70ms/step
[[0.02517079]
 [0.07287466]
 [0.04927137]
 [0.2469862 ]]


In [161]:
prediction_prob = pd.DataFrame(prediction,columns=['prediction_prob']).astype('float')
prediction_prob 

,prediction_prob
0,0.025171
1,0.072875
2,0.049271
3,0.246986


In [162]:
for index, row in prediction_prob.iterrows():
    if row['prediction_prob']> 0.5:
        print('Person is likely to churn')
    else:
        print('Churn possibility is low')

Churn possibility is low
Churn possibility is low
Churn possibility is low
Churn possibility is low
